<a href="https://colab.research.google.com/github/kushal-h/donut_hotel_menu/blob/main/Hotel_menu/Hotel_menu_donut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Donut package

In [ ]:
!git clone https://github.com/clovaai/donut.git
!cd donut && pip install .

In [ ]:
!unzip hotel_menu.zip

Splitting test,train,Validation data

In [ ]:
import os
import json
import shutil
from tqdm.notebook import tqdm
lines = []
images = []
for ann in tqdm(os.listdir("/content/hotel_menu/Json files/")[:30]):
  if ann != ".ipynb_checkpoints":
    with open("/content/hotel_menu/Json files/" + ann) as f:
      data = json.load(f)
    images.append(ann[:-4] + "png")
    line = {"gt_parse": data}
    lines.append(line)

with open("./hotel_donut/train/metadata.jsonl", 'w+') as f:
  for i, gt_parse in enumerate(lines):
    print(images[i])
    line = {"file_name": images[i], "ground_truth": json.dumps(gt_parse)}
    f.write(json.dumps(line) + "\n")
    
    shutil.copyfile("/content/hotel_menu/Png files/" + images[i], "/content/hotel_donut/train/" + images[i])

In [ ]:
!cd donut && python train.py --config config/train_hotel.yaml

2023-04-27 23:55:00.765215: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
resume_from_checkpoint_path: None
result_path: ./result
pretrained_model_name_or_path: naver-clova-ix/donut-base
dataset_name_or_paths: 
  - /content/hotel_donut
sort_json_key: False
train_batch_sizes: 
  - 1
val_batch_sizes: 
  - 1
input_size: 
  - 1280
  - 960
max_length: 768
align_long_axis: False
num_nodes: 1
seed: 2022
lr: 3e-05
warmup_steps: 300
num_training_samples_per_epoch: 800
max_epochs: 30
max_steps: -1
num_workers: 8
val_check_interval: 1.0
check_val_every_n_epoch: 10
gradient_clip_val: 1.0
verbose: True
exp_name: train_hotel
exp_version: 20230427_235504
Config is saved at result/train_hotel/20230427_235504/config.yaml
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `li

In [ ]:
from donut import DonutModel
from PIL import Image
import torch
model = DonutModel.from_pretrained("/content/donut/result/train_hotel/20230427_235504")
if torch.cuda.is_available():
    model.half()
    device = torch.device("cuda")
    model.to(device)
else:
    model.encoder.to(torch.bfloat16)
model.eval()
image = Image.open("/content/hotel_donut/test/012.png").convert("RGB")
output = model.inference(image=image, prompt="<s_sroie-donut>")
output

{'predictions': [[{'price': '$18.00'},
   {'food name': 'Gongura Chicken Curry', 'price': '$18.00'},
   {'food name': 'Chicken Tikka Masala', 'price': '$18.00'},
   {'food name': 'Lamb Saag', 'price': '$20.00'},
   {'food name': 'Chicken Korma', 'price': '$18.00'},
   {'food name': 'Lamb Rogan Josh', 'price': '$20.00'},
   {'food name': 'Chicken Chettinad', 'price': '$18.00'},
   {'food name': 'Lamb Korma', 'price': '$20.00'}]]}

In [ ]:
!cd ./donut && python test.py --dataset_name_or_path ../hotel_donut --pretrained_model_name_or_path /content/donut/result/train_hotel/20230427_235504 --save_path ./result/train_hotel/output.json

2023-04-28 00:07:26.136916: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Resolving data files: 100% 31/31 [00:00<00:00, 77811.74it/s]
Found cached dataset imagefolder (/root/.cache/huggingface/datasets/imagefolder/hotel_donut-3e64f39d891ff108/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
100% 5/5 [00:04<00:00,  1.20it/s]
Total number of samples: 5, Tree Edit Distance (TED) based accuracy score: 0.8964204529086561, F1 accuracy score: 0.8863636363636364
